In [1]:
from dotenv import load_dotenv
import os
import sys
sys.path.append("..")

In [2]:
load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from typing import List, Dict, Any, Optional
import json

In [4]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.
)

In [5]:
from langchain_core.prompts import ChatPromptTemplate

# Định nghĩa prompt template với rõ vai trò system và user
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """You are a professional developer with experience in writing SPARQL for ontology file. Your task is to create a plan to transform the provided natural query to SPARQL. Please follow the detailed instruction below:
- If a query need to compute or find out the interval of time, divide query into simple natural queries and merge queries in last step. Else please not change the query.
- Add level for each step: **simple**: can generate query immediately and **complex**: must use previous queries.
- Add type of SPARQL query for each step: SELECT, ASK, DESCRIBE and CONSTRUCT.
- Do not create SPARQL query.
- If can not create plan to transform to SPARQL, the output is []
The output format must be following this format: 
[{{"step": "step query", "sparql_type": "SELECT or ASK or DESCRIBE or CONSTRUCT", "level": "simple or complex"}}, ...]"""
    ),
    ("user", "Câu query: {user_query}. Old plan and feedback: {feedback}")
])

# Định nghĩa input
user_query = "Vua Quang Trung mất ngày nào"
feedback = "[]"

# Format ra danh sách message
messages = prompt.format_messages(user_query=user_query, feedback=feedback)

In [22]:
messages

[SystemMessage(content='You are a professional developer with experience in writing SPARQL for ontology file. Your task is to create a plan to transform the provided natural query to SPARQL. Please follow the detailed instruction below:\n- If a query need to compute or find out the interval of time, divide query into simple natural queries and merge queries in last step. Else please not change the query.\n- Add level for each step: **simple**: can generate query immediately and **complex**: must use previous queries.\n- Add type of SPARQL query for each step: SELECT, ASK, DESCRIBE and CONSTRUCT.\n- Do not create SPARQL query.\n- If can not create plan to transform to SPARQL, the output is []\nThe output format must be following this format: \n[{"step": "step query", "sparql_type": "SELECT or ASK or DESCRIBE or CONSTRUCT", "level": "simple or complex"}, ...]', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Câu query: Vua Quang Trung mất ngày nào. Old plan and feedba

In [23]:
ans = llm.invoke(messages)

In [24]:
ans.content

'[{"step": "Find the death date of Vua Quang Trung", "sparql_type": "SELECT", "level": "simple"}]'

In [17]:
json.loads(ans.content)

[]

In [4]:
class PlanFormulationAgent:
    """
    Slave agent responsible for creating execution plans for SPARQL queries.
    Formulates plans for complex scenarios requiring multiple queries or 
    processing steps.
    """
    def __init__(self):
        """
        Initialize the plan formulation agent
        """
    
        self.agent = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.
        )
    
        self.num_retry = 2

    def _prepare_plan_prompt(self, user_query: str, feedback: Optional[str] = None) -> List[Any]:
        """
        Create prompt for planning based on user query
        
        Args:
            user_query: Natural user query 
        
        Returns:
            Execution plan as a list of dictionary
        """
        prompt = ChatPromptTemplate.from_messages([
            ("system", 
             """You are a professional developer with experience in writing SPARQL for ontology file. Your task is to create a plan to transform the provided natural query to SPARQL. Please follow the detailed instruction below:
        - If a query need to compute or find out the interval of time, divide query into simple natural queries and merge queries in last step. Else please not change the query.
        - Add level for each step: **simple**: can generate query immediately and **complex**: must use previous queries.
        - Add type of SPARQL query for each step: SELECT, ASK, DESCRIBE and CONSTRUCT.
        - Do not create SPARQL query.
        - If can not create plan to transform to SPARQL, the output is []
        The output format must be following this format: 
        [{{"step": "step query", "sparql_type": "SELECT or ASK or DESCRIBE or CONSTRUCT", "level": "simple or complex"}}, ...]"""
            ),
            ("user", "{user_query}{feedback}")
        ])
        if feedback is not None:
            feedback = ". Old plan and feedback: {}".format(str(feedback))
        else:
            feedback = ""
        
        return prompt.format_messages(user_query=user_query, feedback=feedback)

    def formulate_plan(
        self, 
        refined_query: str,
        mapped_entities: Optional[Dict[str, Any]] = None, 
        ontology_info: Optional[Dict[str, Any]] = None,
        validation_feedback: Optional[str] = None
    ) -> List[Dict[str, Any]]:
        """
        Formulate an execution plan for SPARQL queries
    
        Args:
            refined_query: The refined user query
            mapped_entities: Dictionary of mapped ontology entities
            ontology_info: Information about the ontology structure
            validation_feedback: Optional feedback from validation agent
    
        Returns:
            Execution plan as list
        """
        
        for i in range(self.num_retry):
            try:
                prompt = self._prepare_plan_prompt(refined_query)
                plan = self.agent.invoke(prompt)
                plan = json.loads(plan.content)
                break
            except Exception as e:
                print(e)
                plan = None
                continue 
    
        return plan

In [5]:
planner = PlanFormulationAgent()

In [9]:
user_query = "What are all the subclasses of Person?"

In [10]:
plan = planner.formulate_plan(refined_query=user_query)

In [11]:
plan

[{'step': 'Find all subclasses of Person',
  'sparql_type': 'SELECT',
  'level': 'simple'}]

In [12]:
!pip list

Package                      Version         Editable project location
---------------------------- --------------- -------------------------
absl-py                      2.1.0
accelerate                   0.27.0
addict                       2.4.0
aiofiles                     23.2.1
aiohttp                      3.8.6
aiosignal                    1.3.1
albumentations               1.4.2
altair                       5.4.1
annotated-types              0.7.0
antlr4-python3-runtime       4.9.3
anyio                        4.3.0
appdirs                      1.4.4
argon2-cffi                  23.1.0
argon2-cffi-bindings         21.2.0
arrow                        1.3.0
asttokens                    2.4.1
async-lru                    2.0.4
async-timeout                4.0.3
attrs                        23.1.0
av                           14.2.0
Babel                        2.14.0
basicsr                      1.4.2
beautifulsoup4               4.12.3
bitsandbytes                 0.41.3
bleach   